In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from numpy import random

from ipywidgets import *
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

### Admissions

In [2]:
def plot_hrly_avgs(sliders, label, color):
    ind = np.arange(24)  
    width = 0.75  

    fig, ax = plt.subplots(1, figsize=(12, 5))
    rects1 = ax.bar(ind, [i.value for i in sliders], width, label=label, color=color)

    ax.set_ylabel('avg # of patients')
    ax.set_xlabel('hour of day')

    ax.set_xticks(ind)
    ax.legend()
    plt.show()

def hrly_avg_change(b):
    output1.clear_output()
    with output1:
        plot_hrly_avgs(avg_pl_hrly_sliders, "Planned", "b")
    
    output2.clear_output()
    with output2:
        plot_hrly_avgs(avg_upl_hrly_sliders, "Unplanned", "r")

In [3]:
avg_plan_hrly = np.array([0.2, 0.2, 0.2, 0.2, 0.2, 1.5, 2.0, 2.1, 2.2, 2.2, 3.2, 3.1, 2.1, 2.1, 2.0, 1.5, 1.4, 1.1, 1.0, 1.0, 
                             0.2, 0.2, 0.2, 0.2])

avg_pl_hrly_sliders = [FloatSlider(value=n, min=0.0, max=20.0, step=0.25,  description=str(h), continuous_update=False, 
                                   orientation="vertical", layout=Layout(width="110%")) for n,h in zip(avg_plan_hrly, np.arange(0,24, 1))]

avg_unplan_hrly = np.array([3.5, 3.1 ,2.9, 2.8, 2.64, 2.45, 2.4, 4.5, 5.9, 6.7, 6.5, 5.4, 5.6, 5.8, 5.7, 5.6, 5.3, 5.2, 5.3, 6,
                         6.5, 5.2, 4.3, 3.9])

avg_upl_hrly_sliders = [FloatSlider(value=n, min=0.0, max=30.0, step=0.25,  description=str(h), continuous_update=False, 
                                    orientation="vertical", layout=Layout(width="100%")) for n,h in zip(avg_unplan_hrly, np.arange(0,24, 1))]

In [4]:
output1 = Output(wait=True)
output2 = Output()

output1.layout.height = '350px'
output2.layout.height = '350px'

_ = [i.observe(hrly_avg_change, names=['value']) for i in avg_pl_hrly_sliders]
_ = [i.observe(hrly_avg_change, names=['value']) for i in avg_upl_hrly_sliders]

In [5]:
center_layout = Layout(align_items="center", justify_content='center')

In [6]:
tabs = [VBox([output1, HBox(avg_pl_hrly_sliders)], layout=center_layout), 
      VBox([output2, HBox(avg_upl_hrly_sliders)], layout=center_layout)]

plt_tab = Tab()
plt_tab.children = tabs
plt_tab.set_title(0, 'Planned Adm Hrly')
plt_tab.set_title(1, 'Unplanned Adm Hrly')

In [7]:
AppLayout(left_sidebar=VBox([HTML(value="<b>Admission Statistics:</b><p>Set hourly averages for gross planned and unplanned admissions </p>")]),
          center=plt_tab)

AppLayout(children=(VBox(children=(HTML(value='<b>Admission Statistics:</b><p>Set hourly averages for gross pl…

In [8]:
hrly_avg_change(None)

In [9]:
def p_los_gen(x):
    times = []
    for i in range(x['Unplanned']):
        times.append(x['Datetime'] + pd.Timedelta(value=abs(round(random.normal(loc=pl_los_avg.value, scale=pl_los_std.value))), unit='hours' ))
    return times
    
def up_los_gen(x):
    times = []
    for i in range(x['Planned']):
        times.append(x['Datetime'] + pd.Timedelta(value=abs(round(random.normal(loc=unpl_los_avg.value, scale=unpl_los_std.value))), unit='hours' ))
    return times 

def gen_los(df):
    df['Admit'] = df['Planned'] + df['Unplanned']
#     df = df.reset_index()
#     df.rename(columns={'index': 'Datetime'}, inplace=True)
    df['Datetime'] = df.index
    
    df['Discharge Time'] = df.apply(p_los_gen, axis=1) + df.apply(up_los_gen, axis=1)
    df_los = df[['Datetime','Discharge Time']].explode('Discharge Time').dropna().reset_index(drop=True)
    
    dis = []
    
    for time in df['Datetime']:
        dis.append(len(df_los['Discharge Time'][df_los['Discharge Time'] == time].tolist()))
        
    df['Discharge'] = dis
    df.drop(columns=['Discharge Time'], inplace=True)
    
    return df

In [10]:
def count_days(start, end):
    if start.value and end.value:
        delta = end.value - start.value
        return delta.days

def gen_adm_df(avg_p_sliders, avg_up_sliders, start_wid, end_wid, back=21):

    num_days = count_days(start_wid, end_wid)
    backdate = start.value - datetime.timedelta(days=back)
    datetime_index = [pd.Timestamp((backdate + datetime.timedelta(days=day)).strftime("%Y-%m-%d") + f' {hr}:00:00') for day in range(num_days+back) for hr in range(24)]

    df = pd.DataFrame(columns=["ED Volume", "Unplanned", "Planned"], index=datetime_index)

    df['Planned'] = [(random.poisson(p.value)) for days in range(num_days+21) for p in avg_p_sliders]
    df['ED Volume'] = [round(random.poisson(up.value)) for days in range(num_days+21) for up in avg_up_sliders]
    df['Unplanned'] = round(df['ED Volume'] * unplan_adm_rate.value).astype(int)
    
    df = gen_los(df)
    
    df['Occupied'] = df['Admit'].cumsum() - df['Discharge'].cumsum()
    
    return df

In [11]:
#### Dates selectors ####
today = datetime.date.today()

start = DatePicker( description='Begin:', disabled=False)
start.value = today - datetime.timedelta(days=7)

end = DatePicker(description='End:', disabled=False)
end.value = today

unplan_adm_rate = FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='(%)', continuous_update=False)

In [12]:
#### Sliders #### 

num_beds = IntSlider(value=250, min=100, max=500, description='Beds',  continuous_update=False)

pl_los_avg = FloatSlider(value=72.0, min=0.0, max=240.0, step=1.0,  description='avg', continuous_update=False)
pl_los_std = FloatSlider(value=36.0, min=0.0, max=100.0, step=1.0,  description='std', continuous_update=False)

unpl_los_avg = FloatSlider(value=48.0, min=0.0, max=120.0, step=1.0,  description='avg', continuous_update=False)
unpl_los_std = FloatSlider(value=24.0, min=0.0, max=60.0, step=1.0,  description='std', continuous_update=False)

In [13]:
def autolabel(ax, rects, xpos='center'):
    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

In [14]:
def plot_adms(df):
    df = df[(df['Datetime'] > pd.Timestamp(start.value))]
    ind = np.arange(df['Datetime'].dt.day.nunique())
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(ind - width/2, df['Planned'].resample('D').sum().tolist(), width, label='Planned')
    rects2 = ax.bar(ind + width/2, df['Unplanned'].resample('D').sum().tolist(), width, label='Unplanned')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('# of patients')
    ax.set_title('Admissions Per Day')
    ax.set_xticks(ind)
    ax.set_xticklabels(np.unique(df['Datetime'].dt.date))
    fig.autofmt_xdate()
    ax.legend()

    autolabel(ax, rects1, "left")
    autolabel(ax, rects2, "right")

    plt.show()

In [15]:
def plot_ed_vol(df):
    df = df[(df['Datetime'] > pd.Timestamp(start.value))]

    fig, ax = plt.subplots(figsize=(18, 7))
    bars = ax.bar(df['Datetime'], df['ED Volume'], label='ED Volume',  align='center', width=0.02)
    
    for bar in bars:
        bar.set_edgecolor("black")
        bar.set_linewidth(1)


    ax.set_ylabel('# of patients')
    ax.set_title('ED Volume by hour')
#     ax.set_xticks(ind)
#     ax.set_xticklabels(np.unique(df.index.date))
    fig.autofmt_xdate()
    ax.legend()

    plt.show()

In [16]:
def plot_beds(df):
    df = df[(df['Datetime'] > pd.Timestamp(start.value))]
    max_beds = float(num_beds.value)
    fig, ax = plt.subplots(1, figsize=(15,6))
    ax.plot(df['Datetime'], df['Occupied'])
    plt.axhline(y=max_beds, color='r', linestyle='-')
    ax.set_ylabel('# of patients')
    ax.fill_between(df['Datetime'], df['Occupied'], max_beds, where= df['Occupied'] >= max_beds, alpha=0.50,  facecolor='r', edgecolor='r', interpolate=True)
#     ax.set_ylim(100.0, df['Occupied'].max() + 25)
    fig.autofmt_xdate()
    ax.legend(['Total Beds Occupied', "Max Bed Capacity"])
    plt.show();
   

In [18]:
def plot_beds_util(df):
    df = df[(df['Datetime'] > pd.Timestamp(start.value))]
    max_beds = float(num_beds.value)
    fig, ax = plt.subplots(1, figsize=(15,6))
    percents = (df['Occupied']/max_beds)*100
    ax.plot(df['Datetime'], percents)
    plt.axhline(y=100, color='r', linestyle='-')
    ax.set_ylabel('% of occupied beds')
    ax.set_ylim(0.0, 150)
    ax.fill_between(df['Datetime'], percents, max_beds, where= percents >= max_beds, alpha=0.50,  facecolor='r', edgecolor='r', interpolate=True)
    fig.autofmt_xdate()
    ax.legend(["% of Bed Utilization"])
    plt.show();
   

In [17]:
# df = gen_adm_df(avg_pl_hrly_sliders, avg_upl_hrly_sliders, start, end)

In [19]:
def bed_capacity_matrix(start, end, df1, df2, hours_back=24):
    
    matrix_df = pd.DataFrame() 
    num_days = count_days(start, end)
    count=0
    df = df1.append(df2, ignore_index=True).sort_values('admit')
    
    # for each day in date range
    for day in range(num_days):
        current_day = start.value + timedelta(days=day)
        
        # for each hour in 24 hour day
        for hr in range(24):
            
            col = str(hr)+current_day.strftime(" %m-%d")
            matrix_df.insert(count, col, None)
            count+=1
            current_hr = datetime(year=current_day.year, month=current_day.month, 
                                  day=current_day.day, hour=0, minute=0, second=0)  + timedelta(hours=hr)
            patients = []
            
            # go hours back 
            for i in range(hours_back): 
                past = pd.Timestamp((current_hr - timedelta(hours=i)).strftime("%m-%d-%Y %H:%M:%S"))
                patients.append(len(df[(df['admit']<=past)]) - len(df[(df['discharge']<=past)]))
                
           
            matrix_df[col] = patients
                
            
    return matrix_df

In [20]:
def on_change(b):
    df = gen_adm_df(avg_pl_hrly_sliders, avg_upl_hrly_sliders, start, end)
    
    output3.clear_output()
    with output3:
        plot_adms(df)
    
    output4.clear_output()
    with output4:
        plot_ed_vol(df)
        
    df = gen_los(df)
    
    output5.clear_output()
    with output5:
        plot_beds(df)
    
    output6.clear_output()
    with output6:
        plot_beds_util(df)
    

In [21]:
gen_button = Button(description="Generate Data")

output3 = Output()
output4 = Output()
output5 = Output()
output6 = Output()

output3.layout.height = '450px'
output4.layout.height = '450px'
output5.layout.height = '450px'
output6.layout.height = '450px'

gen_button.on_click(on_change)
start.observe(on_change, names=['value'])
end.observe(on_change, names=['value'])
unplan_adm_rate.observe(on_change, names=['value'])


num_beds.observe(on_change, names=['value'])
unpl_los_avg.observe(on_change, names=['value'])
unpl_los_std.observe(on_change, names=['value'])
pl_los_avg.observe(on_change, names=['value'])
pl_los_std.observe(on_change, names=['value'])

In [24]:
tabs2 = [HBox([output3], layout=center_layout), HBox([output4], layout=center_layout)]
tab2 = Tab()
tab2.children = tabs2
tab2.set_title(0, 'Admissions Per Day')
tab2.set_title(1, 'ED Volume')

In [25]:
l_siderbar = VBox([HTML(value="<b>Unplanned Admission Rate:</b>"),unplan_adm_rate ])
dates = HBox([start, end], layout=Layout(justify_content='center', align_items='center'))
footer = HBox([gen_button], layout=center_layout)

In [26]:
AppLayout(header=dates,
          left_sidebar=l_siderbar,
          center=tab2,
          footer=footer,
          justify_content='center', 
          align_items='center')

AppLayout(children=(HBox(children=(DatePicker(value=datetime.date(2020, 9, 8), description='Begin:'), DatePick…

### Bed Capacity

In [27]:
tabs3 = [HBox([output5], layout=center_layout), HBox([output6], layout=center_layout)]
tab3 = Tab()
tab3.children = tabs3
tab3.set_title(0, 'Occupied Beds')
tab3.set_title(1, 'Bed Utilization')

In [28]:
los_sidebar = VBox([HTML(value="<b>Total Beds:</b>"), num_beds,
                    HTML(value="<b>Planned LOS:</b>"), pl_los_avg, pl_los_std, 
                    HTML(value="<b>Unplanned LOS:</b>"), unpl_los_avg, unpl_los_std])

In [29]:
AppLayout(left_sidebar=los_sidebar, 
         center=tab3)

AppLayout(children=(VBox(children=(HTML(value='<b>Total Beds:</b>'), IntSlider(value=275, continuous_update=Fa…

In [30]:
on_change(None)